# 네이버뉴스_크롤링

# 생활/문화, 스포츠  
  
## make_df 사용자 함수

In [1]:
def make_df(topic):
    import requests
    from bs4 import BeautifulSoup
    import pandas as pd
    from datetime import datetime

    time_li = []
    time_list = [] # 순서정렬용
    media_li = []
    title_li = []
    document_li = []

    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/90.0.4430.93 Safari/537.36"}

    if topic == "생활/문화":
        url = 'https://news.naver.com/main/main.naver?mode=LSD&mid=shm&sid1=103'
        req = requests.get(url, headers=headers)
        html = req.text
        soup = BeautifulSoup(html, 'html.parser')
        contents = soup.find_all(class_='cluster_group _cluster_content')

        for content in contents[:4]:
            link = content.find(class_='cluster_text_headline nclicks(cls_lif.clsart)')['href']
            req = requests.get(link, headers=headers)
            html = req.text
            soup = BeautifulSoup(html, 'html.parser')
            time = soup.select('span.t11')[-1].text
            time_li.append(time) # time

            time = time.replace('오전','AM').replace('오후','PM')
            time = datetime.strptime(time, '%Y.%m.%d. %p %I:%M') # datetime 으로 파싱
            time_list.append(time) # 순서정렬용
            media_li.append(content.find(class_='cluster_text_press').text) # media
            title_li.append(soup.title.text.split(':')[0]) # title
            document_li.append(soup.find(id='articleBodyContents').text.strip('\n동영상 뉴스\t')) # document

        df = pd.DataFrame({'time_list':time_list,'time':time_li,'media':media_li,'title':title_li,'document':document_li})
        df.sort_values(by='time_list',ascending=False,inplace=True) # (시간 기준) 최신 순으로 정렬
        df.drop(['time_list'],axis=1,inplace=True) # 필요없는 컬럼 삭제
        
    elif topic == "스포츠":
        url = 'https://sports.news.naver.com'
        req = requests.get(url, headers=headers)
        html = req.text
        soup = BeautifulSoup(html, 'html.parser')
        contents = soup.find_all(class_='today_item')

        for content in contents[:4]:
            link = url+content.find(class_='link_today')['href']  
            req = requests.get(link, headers=headers)
            html = req.text
            soup = BeautifulSoup(html, 'html.parser')

            time = soup.find(class_='info').text.split('최종수정')[1].split('기사원문')[0].strip() # time
            time_li.append(time)
            time = time.replace('오전','AM').replace('오후','PM')
            time = datetime.strptime(time, '%Y.%m.%d. %p %I:%M') # datetime 으로 파싱
            time_list.append(time) # 순서정렬용
            media_li.append(content.find(class_='information').text.split('\n')[1]) # media
            title_li.append(content.find(class_='title').text) # title
            document_li.append(soup.find(id='newsEndContents').text.split('기사제공')[0].strip('\n')) # document

        df = pd.DataFrame({'time_list':time_list,'time':time_li,'media':media_li,'title':title_li,'document':document_li})
        df.sort_values(by='time_list',ascending=False,inplace=True) # (시간 기준) 최신 순으로 정렬
        df.drop(['time_list'],axis=1,inplace=True) # 필요없는 컬럼 삭제
        
    return df
make_df('사회/문화')

,time,media,title,document
1,2021.11.23. 오후 2:17,한국경제,"중국, '오징어게임' 이어 '지옥'까지 훔쳐보네",넷플릭스 서비스 안되는 중국'지옥공사' 이름으로 9600만 클릭/사진=넷플릭스 '지...
3,2021.11.23. 오후 12:34,한국경제,아우디가 만든 '아이언맨 차' 서울모빌리티쇼에 선보인다,e-트론 GT 등 18대 차량 전시아우디 e-트론 GT. 사진=아우디 제공.아우디는...
2,2021.11.23. 오전 10:09,한국경제,"""내비 불편"" 지적받던 지프, T맵 넣었다…'그랜드 체로키 L' 출시",아시아 최초 출시…지프 브랜드 첫 '3열 SUV'올 뉴 그랜드 체로키 L 써밋 리저...
0,2021.11.23. 오전 9:14,머니투데이,스파이더맨에 '아이오닉 5·투싼' 나온다..현대차-소니 첫 작품,아이오닉 5와 스파이더맨/사진제공=현대차현대자동차의 첫 전용전기차 '아이오닉 5'와...
